# Visualize CNN Weights and Activations

In [1]:
import math
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import skimage.io
import skimage.transform
import skimage.filters
import keras
from keras.applications import vgg16
from keras import backend as K

# Allow graph embeding in notebook
%matplotlib inline

2022-09-08 15:30:06.219674: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-08 15:30:06.333685: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-09-08 15:30:06.768365: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.2/lib64:/usr/local/cuda-11.2/lib64
2022-09-08 15:30:06.768411: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: 

## Helper Functions

In [2]:
def tensor_summary(tensor):
    """Display shape, min, and max values of a tensor."""
    print("shape: {}  min: {}  max: {}".format(tensor.shape, tensor.min(), tensor.max()))

    
def normalize(image):
    """Takes a tensor of 3 dimensions (height, width, colors) and normalizes it's values
    to be between 0 and 1 so it's suitable for displaying as an image."""
    image = image.astype(np.float32)
    return (image - image.min()) / (image.max() - image.min() + 1e-5)


def display_images(images, titles=None, cols=5, interpolation=None, cmap="Greys_r"):
    """
    images: A list of images. I can be either:
        - A list of Numpy arrays. Each array represents an image.
        - A list of lists of Numpy arrays. In this case, the images in
          the inner lists are concatentated to make one image.
    """
    titles = titles or [""] * len(images)
    rows = math.ceil(len(images) / cols)
    height_ratio = 1.2 * (rows/cols) * (0.5 if type(images[0]) is not np.ndarray else 1)
    plt.figure(figsize=(15, 15 * height_ratio))
    i = 1
    for image, title in zip(images, titles):
        plt.subplot(rows, cols, i)
        plt.axis("off")
        # Is image a list? If so, merge them into one image.
        if type(image) is not np.ndarray:
            image = [normalize(g) for g in image]
            image = np.concatenate(image, axis=1)
        else:
            image = normalize(image)
        plt.title(title, fontsize=9)
        plt.imshow(image, cmap=cmap, interpolation=interpolation)
        i += 1


## Build VGG16 Convolutional Network

In [3]:
# Build a VGG16 Convolutional Network pre-trained on ImageNet
model = vgg16.VGG16(weights='imagenet')
model.summary()

2022-09-08 15:30:08.123108: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-08 15:30:08.497461: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.2/lib64:/usr/local/cuda-11.2/lib64
2022-09-08 15:30:08.497478: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1934] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2022-09-08 15:30:08.497682: I tensorflow/core/platform/cpu_feature_guard.cc:193] This Tensor

553467096/553467096 [==============================] - 325s 1us/step
Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                          

## Test the Model on Random Images

In [4]:
# Pick a random image from the Web.
# Make sure it's 224x224 image since VGG16 expects this size.
image = skimage.io.imread("http://lorempixel.com/224/224/animals/")
assert image.shape == (224, 224, 3), "Must be 224x224 RGB Image"
tensor_summary(image)
display_images([image], cols=2)

URLError: <urlopen error [Errno 110] Connection timed out>

In [ ]:
# Convert image to float
x = image.astype(np.float32)
# Make it a batch of one. The model expects a batch, not a single image
x = x[np.newaxis,...]
# Preprocess image. Convert RGB to BGR and subtract the ImageNet mean.
x = vgg16.preprocess_input(x)

In [ ]:
# Classify the image
predictions = model.predict(x)
# We'll get a 1000 values. Print the first 10.
print(predictions[0][:10])

In [ ]:
# Find the largest confidence value. This corresponds to the label index.
label_index = np.argmax(predictions)
print("label index: ", label_index)
# Display the top 5 classes
vgg16.decode_predictions(predictions)

In [ ]:
# List of ImageNet classes. Print a subset of the 1000
# The previous call to decode_predictions() sets the value
# of CLASS_INDEX, so call that function first.
imagenet_classes = keras.applications.imagenet_utils.CLASS_INDEX
for i in range(10):
    print(i, imagenet_classes[str(i)][1])

## Heatmap

In [ ]:
step = 56
heatmap_x = []
for row in range(0, image.shape[0], step):
    for col in range(0, image.shape[1], step):
        new_image = image.copy()
        # Add a square patch. Using a bright color here to make it easier to see.
        new_image[row:row+step, col:col+step, :] = [250,128,128]
        heatmap_x.append(new_image)
heatmap_x = np.stack(heatmap_x)
heatmap_x.shape

In [ ]:
display_images(heatmap_x[:28], cols=8)

In [ ]:
heatmap_y = model.predict(vgg16.preprocess_input(heatmap_x.astype(np.float32)))
tensor_summary(heatmap_y)

In [ ]:
probs = heatmap_y[:, label_index]
tensor_summary(probs)

In [ ]:
heatmap = (probs.max() - probs) / (probs.max()-probs.min())
heatmap = np.reshape(heatmap, (4, 4))
tensor_summary(heatmap)

In [ ]:
tensor_summary(heatmap)
_ = plt.imshow(heatmap, cmap=plt.cm.Reds)

In [ ]:
def apply_mask(image, mask):
    # Resize mask to match image size
    mask = skimage.transform.resize(normalize(mask), image.shape[:2])[:,:,np.newaxis].copy()
    # Apply mask to image
    image_heatmap = image * mask
    tensor_summary(image_heatmap)
    display_images([image_heatmap], cols=2)


# Apply mask to image
apply_mask(image, heatmap**2)

## Visualize the Weights

In [ ]:
weights = model.get_layer("block1_conv1").get_weights()[0]
weights.shape, weights.min(), weights.max()

In [ ]:
display_images([weights[:,:,::-1,i] for i in range(64)], cols=16, interpolation="none")

## Visualize Activations

In [ ]:
def read_layer(model, x, layer_name):
    """Return the activation values for the specifid layer"""
    # Create Keras function to read the output of a specific layer
    get_layer_output = K.function([model.layers[0].input], [model.get_layer(layer_name).output])
    outputs = get_layer_output([x])[0]
    tensor_summary(outputs)
    return outputs[0]
    
def view_layer(model, x, layer_name, cols=5):
    outputs = read_layer(model, x, layer_name)
    display_images([outputs[:,:,i] for i in range(10)], cols=cols)

In [ ]:
view_layer(model, x, "block1_conv1")

In [ ]:
view_layer(model, x, "block1_conv2")

In [ ]:
view_layer(model, x, "block2_conv1")

In [ ]:
view_layer(model, x, "block3_conv1")

In [ ]:
view_layer(model, x, "block4_conv1")

In [ ]:
view_layer(model, x, "block5_conv3")

In [ ]:
a = read_layer(model, x, "block5_conv3")
apply_mask(image, a[:,:, 0])

In [ ]:
# The last layer is a 1D 1000-vector. Visualize as a bar chart.
a = read_layer(model, x, "predictions")
_ = plt.plot(a)

## Gradient Descent to Image Space

Or, gradient ascent, depending on which loss function you choose.

In [ ]:
def build_backprop(model, loss):
    # Gradient of the input image with respect to the loss function
    gradients = K.gradients(loss, model.input)[0]
    # Normalize the gradients
    gradients /= (K.sqrt(K.mean(K.square(gradients))) + 1e-5)
    # Keras function to calculate the gradients and loss
    return K.function([model.input], [loss, gradients])

### Visualize the Gradients on the Input Image

In [ ]:
# Loss function that optimizes one class
loss_function = K.mean(model.get_layer("predictions").output[:,label_index])

# Backprop function
backprop = build_backprop(model, loss_function)

# Calculate gradients on the input image
loss, grads = backprop([x])
tensor_summary(grads)

# Visualize the gradients
grad_image = normalize(grads)
display_images([np.sum(grad_image[0], axis=2), image], cols=2)

## Generate an Image That Maximizes A Class 

In [ ]:
# Start with a random image
random_image = np.random.random((1, 224, 224, 3))
display_images(random_image, cols=2)

In [ ]:
# Iteratively apply gradient ascent
for i in range(50):
    loss, grads = backprop([random_image])
    
    # Multiply gradients by the learning rate and add to the image
    # Optionally, apply a gaussian filter to the gradients to smooth
    # out the generated image. This gives better results.
    # The first line, which is commented out, is the native method
    # and the following line uses the filter. Try with both to
    # see the difference.
    #
    # random_image += grads * .1
    random_image += skimage.filters.gaussian(np.clip(grads, -1, 1), 2) 

    # Print loss value
    if i % 10 == 0:
        print('Loss:', loss)

tensor_summary(random_image)
display_images(random_image[...,::-1], cols=2)

In [ ]:
# Classify the image
predictions = model.predict(random_image)
vgg16.decode_predictions(predictions)

## Maximize Activations for a Hidden Filter

In [ ]:
# Loss function that optimizes one class
loss_function = K.mean(model.get_layer("block3_conv2").output[:,:,:,2])

# Backprop function
backprop = build_backprop(model, loss_function)

# Start with a random image
random_image = np.random.random((1, 224, 224, 3)) 

In [ ]:
# Iteratively apply gradient ascent
for i in range(50):
    loss, grads = backprop([random_image])
    grads = np.nan_to_num(grads)  # In case gradients are NaN
    # Apply gradients with or without a gaussian filter
    # random_image += grads * .1
    random_image += skimage.filters.gaussian(np.clip(grads, -1, 1), 2) 
    if i % 10 == 0:
        print('Loss:', loss)

tensor_summary(random_image)
display_images(random_image[...,::-1], cols=2)